# Load Required Packages

In [1]:
import pandas as pd
import numpy as np
import gc
import pickle

In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import HDBSCAN
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Load

In [4]:
Folder = "E:\\datasets\\car\\"

In [8]:
Sample = pd.read_parquet(f"{Folder}SampleEncoded.parq")
Sample

,mileage,engine_displacement,engine_power,price_eur,maker_alfa-romeo,maker_aston-martin,maker_audi,maker_bentley,maker_bmw,maker_chevrolet,...,seat_count_2.0,seat_count_3.0,seat_count_4.0,seat_count_5.0,seat_count_6.0,seat_count_7.0,seat_count_8.0,seat_count_9.0,fuel_type_diesel,fuel_type_gasoline
0,0.777224,0.218001,0.111079,-0.228444,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
1,0.683206,0.218001,-0.411832,-0.363319,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
2,0.110900,0.210865,-0.316757,-0.111166,False,False,False,False,True,False,...,False,False,False,True,False,False,False,False,True,False
3,0.289515,-0.923750,-0.839668,-0.832452,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
4,0.500893,-0.923750,-0.839668,-0.850044,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True


In [7]:
for x in Sample.columns:
    print(x)

mileage
engine_displacement
engine_power
price_eur
maker_alfa-romeo
maker_aston-martin
maker_audi
maker_bentley
maker_bmw
maker_chevrolet
maker_chrysler
maker_citroen
maker_dodge
maker_fiat
maker_ford
maker_honda
maker_hummer
maker_hyundai
maker_infinity
maker_isuzu
maker_jaguar
maker_jeep
maker_kia
maker_lamborghini
maker_lancia
maker_lexus
maker_lotus
maker_maserati
maker_mazda
maker_mercedes-benz
maker_mini
maker_mitsubishi
maker_nissan
maker_opel
maker_peugeot
maker_porsche
maker_renault
maker_rolls-royce
maker_rover
maker_seat
maker_skoda
maker_smart
maker_subaru
maker_suzuki
maker_tesla
maker_toyota
maker_volkswagen
maker_volvo
model_100
model_100-nx
model_1007
model_105
model_106
model_107
model_108
model_115
model_116
model_120
model_121
model_123
model_124
model_125
model_126
model_127
model_128
model_130
model_131
model_140
model_145
model_146
model_147
model_155
model_156
model_159
model_166
model_1800
model_190
model_2
model_200
model_200-sx
model_2008
model_205
model_206
m

In [7]:
X = pd.read_parquet(f"{Folder}X.parq")
y = pd.read_parquet(f"{Folder}y.parq")

# Dimensionality Reduction

In [8]:
pca = PCA(n_components=2).fit(X)

MemoryError: Unable to allocate 19.1 GiB for an array with shape (905, 2834471) and data type object

# Scan for outliers

In [ ]:
dbs = HDBSCAN(min_cluster_size=5,leaf_size=100)

In [ ]:
dbs = dbs.fit(EncodedData.iloc[0:10000])


In [ ]:
dbs.labels_

# Split into Training & Testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.40, random_state=1991
)

if "X" in globals():
    del globals()["X"]

if "y" in globals():
    del globals()["y"]

gc.collect()

# K Neighbors Regressor

In [ ]:
kn = KNeighborsRegressor(25, weights="distance")
kn.fit(X_train, y_train)

In [ ]:
with open("PriceScaler.obj", "rb") as PS_File:
    PriceScaler = pickle.load(PS_File)

In [ ]:
Start = 150000
Number = 500
y_pred = kn.predict(X_test.iloc[Start : Start + Number])

In [ ]:
Comparison = pd.concat(
    [pd.DataFrame(y_pred), y_test.iloc[Start : Start + Number].reset_index(drop=True)],
    axis=1,
)
Comparison = pd.DataFrame(
    PriceScaler.inverse_transform(Comparison), columns=["Predicted", "Price"]
)
print(len(Comparison))
display(Comparison.head())
r2_score(Comparison["Price"], Comparison["Predicted"])

# DBScan

In [ ]:
db = DBSCAN(min_samples=100)
db.fit(X)

# Decision Tree Regressor

In [ ]:
reg = DecisionTreeRegressor().fit(X, y)

In [ ]:
y_pred = reg.predict(X)

In [ ]:
r2_score(y_pred, y)

In [ ]:
# dummies["Y"] = y
# dummies["Pred"] = y_pred
# dummies["Diff"] = (dummies["Y"] - dummies["Pred"]).abs()
# gc.collect()
# Data = pd.read_parquet("clean_df.parq")
# Data["Pred"] = dummies["Pred"]
# Data["Diff"] = dummies["Diff"]
# Data = Data.sort_values(by="Diff", ascending=False)
# Data.to_parquet("PredictionDifferences.parq")
# Data[(Data["maker"] == "volkswagen") & (Data["model"] == "tiguan")]["price_eur"].hist()